In [1]:
import xarray as xr
import pandas as pd
from pathlib import Path
from matplotlib import pyplot as plt
import numpy as np
from pyproj import Transformer
from urllib.parse import quote
from tqdm import tqdm

# Download observation data

In [2]:
obs_raw = pd.read_excel('./data/beetle/artportalen_raw.xlsx',engine="openpyxl", header=2)
print(f'Dataset contains {len(obs_raw)} observations')
obs_raw.head(5)

Dataset contains 1022 observations


,Skyddat fynd,Id,Taxonsorteringsordning,Valideringsstatus,Rödlistade,TaxonId,Artnamn,Vetenskapligt namn,Auktor,Antal,...,Samlingsbeskrivning,Artbestämd av,Bestämningsår,Bekräftad av,Bekräftelseår,Redigeringsansvarig,Rapportör,Observatörer,Länk till BOLD/GenBank,Projektnamn
0,NaN,73125196,8385,Ej granskad,NaN,106570,Granbarkborre,Ips typographus,"(Linnaeus, 1758)",1,...,NaN,NaN,NaN,NaN,NaN,Andreas Garpebring,Andreas Garpebring,"Lars-Ove Wikars, Andreas Garpebring",NaN,NaN
1,NaN,73125909,8385,Ej granskad,NaN,106570,Granbarkborre,Ips typographus,"(Linnaeus, 1758)",1,...,NaN,NaN,NaN,NaN,NaN,Per Ahlgren,Per Ahlgren,"Stefan Björn, Per Ahlgren",NaN,NaN
2,NaN,73158005,8385,Ej granskad,NaN,106570,Granbarkborre,Ips typographus,"(Linnaeus, 1758)",10,...,NaN,NaN,NaN,NaN,NaN,Albin Belsing,Albin Belsing,Albin Belsing,NaN,NaN
3,NaN,73183847,8385,Ej granskad,NaN,106570,Granbarkborre,Ips typographus,"(Linnaeus, 1758)",noterad,...,NaN,NaN,NaN,NaN,NaN,Olof Hedgren,Olof Hedgren,Olof Hedgren,NaN,NaN
4,NaN,73192444,8385,Ej granskad,NaN,106570,Granbarkborre,Ips typographus,"(Linnaeus, 1758)",noterad,...,NaN,NaN,NaN,NaN,NaN,Stefan Björn,Stefan Björn,Stefan Björn,NaN,NaN


NOTES:
- Noggrannhet is the accuracy (m) of the observation's location. Most values are under 2km, which is much smaller than the spatial resolution of weather data. So tracking the accuracy of the observation location is not important.
- Enhet encodes if the amount (Antal) is in individual specimens or colonies. 99.4% of values are not colonies, so empty values and colony observations are mapped to single specimen observations (since it's nearly impossible to accurately map colony to specimens).
- Antal encodes the number of [enhet] observed. 38.1% of values are 'noterad', meaning that an observation was made but the number of specimens was not registered. These values were mapped to 1.

In [3]:
cols = ['Startdatum','Slutdatum', 'Antal', 'Kommun', 'Län', ]

obs = obs_raw[cols].copy()
obs['Startdatum'] = pd.to_datetime(obs['Startdatum'])
obs['Slutdatum'] = pd.to_datetime(obs['Slutdatum'])
 
obs['Duration'] = (obs['Slutdatum'] - obs['Startdatum']).dt.days
obs['Date'] = obs['Startdatum'] + (obs['Slutdatum'] - obs['Startdatum'])/2

obs.loc[obs['Antal'] == 'noterad', 'Antal'] = '1'
obs['Antal'] = pd.to_numeric(obs['Antal'])
obs.head(5)


,Startdatum,Slutdatum,Antal,Kommun,Län,Duration,Date
0,2018-09-14,2018-09-14,1,Robertsfors,Västerbotten,0,2018-09-14
1,2018-09-14,2018-09-14,1,Nybro,Kalmar,0,2018-09-14
2,2018-09-16,2018-09-16,10,Simrishamn,Skåne,0,2018-09-16
3,2018-05-07,2018-08-13,1,Hudiksvall,Gävleborg,98,2018-06-25
4,2018-09-19,2018-09-19,1,Nybro,Kalmar,0,2018-09-19


In [4]:
## Transform from the RT90 coordinate system (csv) to lat/lon
transformer = Transformer.from_crs(
    "EPSG:3021",   # RT90 
    "EPSG:4326",   # WGS84 lat/lon
    always_xy=True # Easting, Northing order
)

obs["Lon"], obs["Lat"] = transformer.transform(
    obs_raw["Ost"].values,
    obs_raw["Nord"].values
)

obs.rename(columns={'Startdatum': 'StartDate', 
                    'Slutdatum':'EndDate', 
                    'Antal':'Quanity', 
                    'Län':'Lan'}, inplace=True)
obs = obs[['Lat', 'Lon', 'Kommun', 'Lan', 'Quanity','Date']]
obs.head(5)

,Lat,Lon,Kommun,Lan,Quanity,Date
0,64.024023,20.650910,Robertsfors,Västerbotten,1,2018-09-14
1,56.729677,15.956413,Nybro,Kalmar,1,2018-09-14
2,55.614954,14.276141,Simrishamn,Skåne,10,2018-09-16
3,61.714395,17.372628,Hudiksvall,Gävleborg,1,2018-06-25
4,56.730931,15.906116,Nybro,Kalmar,1,2018-09-19


In [5]:
obs.to_csv('./data/beetle/artportalen_final.csv')